In [1]:
#pip install - r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import glob

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from time import sleep
from datetime import datetime

import os
from tqdm import tqdm

In [3]:
def consultar(nit,driver):
    #parametro consulta
    url="https://muisca.dian.gov.co/WebRutMuisca/DefConsultaEstadoRUT.faces"
    args={}
    args["DV"]="vistaConsultaEstadoRUT:formConsultaEstadoRUT:dv"
    args["Primer Apellido"]="vistaConsultaEstadoRUT:formConsultaEstadoRUT:primerApellido"
    args["Segundo Apellido"]="vistaConsultaEstadoRUT:formConsultaEstadoRUT:segundoApellido"
    args["Primer Nombre"]="vistaConsultaEstadoRUT:formConsultaEstadoRUT:primerNombre"
    args["Otros Nombres"]="vistaConsultaEstadoRUT:formConsultaEstadoRUT:otrosNombres"
    args["Estado"]="vistaConsultaEstadoRUT:formConsultaEstadoRUT:estado"
    args["Razon Social"]="vistaConsultaEstadoRUT:formConsultaEstadoRUT:razonSocial"


    #consultar
    driver.get(url)
    sleep(0.5)
    elem = driver.find_element(by=By.ID, value="vistaConsultaEstadoRUT:formConsultaEstadoRUT:numNit")
    elem.clear()
    elem.send_keys(str(nit))
    elem.send_keys(Keys.ENTER)
    sleep(0.5)

    record={}
    record["NIT"]=nit
    record["Fecha"]=datetime.today().strftime('%Y-%m-%d')

    for key in args.keys():
        try:
            elem = driver.find_element(by=By.ID, value=args.get(key))
            record[key]=elem.text
        except:
            continue
    
    df_tmp=pd.DataFrame([record])

    file_path = os.path.join("data", "{x}.csv".format(x=nit))
    df_tmp.to_csv(file_path)

    
    return 1

In [4]:
#cargar archivo
file_path = os.path.join("templates", "TEMPLATE.csv")
df=pd.read_csv(file_path)

#abrir entornos de consulta
driver = webdriver.Firefox()

#consultar DIAN
for i in tqdm(range(df.shape[0])):
    if (df.loc[i, 'DOWNLOAD']==0) & (df.loc[i, 'STATE']==1):
        try:
            df.loc[i, 'DOWNLOAD']=consultar(nit=str(df.loc[i, 'NIT']),driver=driver)
            df.to_csv(file_path,index=False)
        except:
            continue

#cerrar consulta
driver.close()

100%|██████████| 9123/9123 [01:57<00:00, 77.89it/s]  


In [5]:
data=[pd.read_csv(i,low_memory=False) for i in tqdm(glob.glob('data/*.csv'))]
df=pd.concat(data).reset_index()
df.to_csv("info.csv",index=False)

100%|██████████| 87/87 [00:00<00:00, 402.85it/s]
